Программа повышения квалификации (научно-педагогических) работников НИУ ВШЭ

# Python для исследователей

*Автор: Алла Тамбовцева, НИУ ВШЭ*  

## Web-scraping: скрэйпинг новостного сайта

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с реального сайта [nplus1.ru](https://nplus1.ru/).

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [4]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [5]:
soup = BeautifulSoup(page.text, 'lxml')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [6]:
# soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [7]:
# print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [8]:
for link in soup.find_all('a'):
    print(link.get('href'))

#
/
#
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/theme/offline
/
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
#
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/theme/offline
https://nplus1.ru/blog/2021/05/13/not-a-soldier
https://nplus1.ru/blog/2021/05/13/not-a-soldier
https://nplus1.ru/blog/2021/05/12/russian-horseman
https://nplus1.ru/blog/2021/05/11/controllability-of-ionization-energy
https://nplus1.ru/blog/2021/05/06/russian-sexual-culture
https://nplus1.ru/blog/2021/04/30/anvisa-vs-sputnikv
https://nplus1.ru/blog/2021/04/30/storm-in-a-teacup
https://nplus1.ru/blog/2021/04/27/ludvig-nobel-och-hans-verk
https://nplus1.ru/blog/2021/04/23/russia-on-the-danube
https://nplus1.ru/blog/2021/04/23/waterquality
https://nplus1.ru/blog/2021/05/14/chnpp-neutron-stream
/news/2021/05/17/upupa-epops
/news/2021/05/17/huntin

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [9]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
urls

['/news/2021/05/17/upupa-epops',
 '/news/2021/05/17/hunting-herding-kenia',
 '/news/2021/05/17/microbiome-health',
 '/news/2021/05/17/thrombosis-hands-covid',
 '/news/2021/05/17/traffic-noise-disrupts-song-learning',
 '/news/2021/05/17/artificial-insemination-in-sharks',
 '/news/2021/05/17/Maagan-Mikhael-B-ship',
 '/news/2021/05/17/heart-structure-cognitive',
 '/news/2021/05/17/inca-sacrifices',
 '/news/2021/05/17/Lalibelachurches',
 '/news/2021/05/17/thermodynamic-cost',
 '/news/2021/05/15/hydromea',
 '/news/2021/05/15/tablet',
 '/news/2021/05/07/silly-walks',
 '/news/2021/05/15/isaac',
 '/news/2021/05/15/electron',
 '/news/2021/05/15/xa100',
 '/news/2021/05/15/dinosaurs-irregular-growth',
 '/news/2021/05/15/proton-mass-radius',
 '/news/2021/05/15/churong-landing',
 '/news/2021/05/15/proton-mass-radius',
 '/news/2021/05/13/z-genome',
 '/news/2021/05/12/pb-neutron-skin',
 '/news/2021/05/14/5qubits-simulates-photon-transport',
 '/news/2021/05/12/antibacterial-microneedles',
 '/news/2021

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки (название сайта) отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [10]:
full_urls = []

for u in urls:
    res = 'https://nplus1.ru' + u
    full_urls.append(res) 

full_urls

['https://nplus1.ru/news/2021/05/17/upupa-epops',
 'https://nplus1.ru/news/2021/05/17/hunting-herding-kenia',
 'https://nplus1.ru/news/2021/05/17/microbiome-health',
 'https://nplus1.ru/news/2021/05/17/thrombosis-hands-covid',
 'https://nplus1.ru/news/2021/05/17/traffic-noise-disrupts-song-learning',
 'https://nplus1.ru/news/2021/05/17/artificial-insemination-in-sharks',
 'https://nplus1.ru/news/2021/05/17/Maagan-Mikhael-B-ship',
 'https://nplus1.ru/news/2021/05/17/heart-structure-cognitive',
 'https://nplus1.ru/news/2021/05/17/inca-sacrifices',
 'https://nplus1.ru/news/2021/05/17/Lalibelachurches',
 'https://nplus1.ru/news/2021/05/17/thermodynamic-cost',
 'https://nplus1.ru/news/2021/05/15/hydromea',
 'https://nplus1.ru/news/2021/05/15/tablet',
 'https://nplus1.ru/news/2021/05/07/silly-walks',
 'https://nplus1.ru/news/2021/05/15/isaac',
 'https://nplus1.ru/news/2021/05/15/electron',
 'https://nplus1.ru/news/2021/05/15/xa100',
 'https://nplus1.ru/news/2021/05/15/dinosaurs-irregular-gro

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [11]:
url0 = full_urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'lxml')

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [12]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="black" name="apple-mobile-web-app-status-bar-style"/>,
 <meta content="7991d7eb02d759f05b9050e111a7e3eb" name="wmail-verification"/>,
 <meta content="2021-05-17" itemprop="datePublished"/>,
 <meta content="Михаил Подрезов" name="mediator_author"/>,
 <meta content="Первые козы и крупно рогатый скот появились в результате обмена около 1180–960 годов до нашей эры" name="description"/>,
 <meta content="Михаил Подрезов" name="author"/>,
 <meta content="" name="copyright"/>,
 <meta content="Древние охотники с кенийского побережья выменяли скот у соседей" property="og:title"/>,
 <meta content="https://nplus1.ru/images/2021/05/17/52df146f53204cb56116c8b2f1b6b3ee.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2021/05/17/hunting-herding-kenia" prope

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [13]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Михаил Подрезов" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [14]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Михаил Подрезов" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [15]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Михаил Подрезов'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [16]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Михаил Подрезов'

Аналогичным образом извлечем дату, заголовок и описание.

In [17]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']

Осталось вытащить рубрики и сложность текста. Если мы посмотрим на исходный код страницы, мы увидим, что нужная нам информация находится в тэгах `<p></p>`:

In [18]:
soup0.find_all('p')

[<p class="table">
 <a data-rubric="archaeology" href="/rubric/archaeology">Археология</a>
 <a data-rubric="history" href="/rubric/history">История</a>
 </p>,
 <p class="table">
 <a href="/news/2021/05/17">
 <time content="2021-05-17" data-unix="1621265849" itemprop="datePublished">
 <span>18:37</span>
 <span>17 Май 2021</span>
 </time>
 </a>
 </p>,
 <p class="table">
 <a href="/difficult/2.7">
 <span>Сложность</span>
 <span class="difficult-value">2.7</span>
 </a>
 </p>,
 <p class="title"></p>,
 <p class="credits">Wikimedia Commons CC</p>,
 <p>Археологи обнаружили самый ранний случай присутствия одомашненных коз и крупного рогатого скота на юго-восточном побережье Кении в железном веке. Они установили, что около 1180–960 годов до нашей эры охотники со стоянки Панга-я-Саиди, вероятно, выменивали скот в небольшом количестве у соседних популяций. Статья <a href="https://linkinghub.elsevier.com/retrieve/pii/S0305440321000388" rel="nofollow" target="_blank">опубликована</a> в <i>Journal of

Выберем из полученного списка первый элемент и найдем в нем все тэги `<a>`:

In [19]:
soup0.find_all('p')[0].find_all('a')

[<a data-rubric="archaeology" href="/rubric/archaeology">Археология</a>,
 <a data-rubric="history" href="/rubric/history">История</a>]

Получился список из одного элемента. Применим списковые включения – вытащим из каждого элемента (их могло бы быть больше) текст и поместим его в новый список `rubrics`.

In [20]:
rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
rubrics

['Археология', 'История']

Осталась только сложность. Возьмем соответствующий кусок кода:

In [21]:
soup0.find_all('span', {'class' : 'difficult-value'})

[<span class="difficult-value">2.7</span>]

И выберем оттуда текст.

In [22]:
diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
diff

'2.7'

Теперь перейдем к тексту самой новости. Как можно заметить, текст сохранен в абзацах `<p></p>`, причем безо всяких атрибутов. Сообщим Python, что нас интересуют куски с пустым атрибутом `class`:

In [23]:
text_list = soup0.find_all('p', {'class' : None})

«Выцепим» все тексты (без тэгов) из полученного списка:

In [24]:
text = [t.text for t in text_list]

Склеим все элементы списка `text` через пробел:

In [25]:
final_text = ' '.join(text)
final_text

'Археологи обнаружили самый ранний случай присутствия одомашненных коз и крупного рогатого скота на юго-восточном побережье Кении в железном веке. Они установили, что около 1180–960 годов до нашей эры охотники со стоянки Панга-я-Саиди, вероятно, выменивали скот в небольшом количестве у соседних популяций. Статья опубликована в Journal of Archaeological Science. Переход человечества от присваивающего хозяйства к производящему, начавшийся примерно 10–12 тысяч лет назад, в некоторых регионах запоздал на многие тысячи лет, несмотря на подходящие природные условия. Так, в восточной Африке распространение скотоводства ученые датируют\xa0примерно пятью тысячами лет до нашей эры. В бассейне озера Виктории, то есть на территории современных Танзании, Кении и Уганды, этот тип хозяйства распространился лишь около 3,3 тысячи лет до нашей эры. Однако многие даже соседние регионы продолжали специализироваться лишь на охоте и собирательстве. Пещерная стоянка Панга-я-Саиди расположена\xa0на тропическо

Все здорово, только мешают отступы-переходы на новую строку `\n`. Заменим их на пробелы с помощью метода `.replace`:

In [26]:
final_text = final_text.replace('\n', ' ')

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [37]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']
    rubrics = ','.join([r.text for r in soup0.find_all('p')[0].find_all('a')])
    diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
    text_list = soup0.find_all('p', {'class' : None})
    text = [t.text for t in text_list]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', ' ')
    
    return url0, date, author, description, title, final_text, rubrics, diff

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [38]:
from time import sleep

In [39]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in full_urls:
    res = GetNews(link)
    news.append(res)
    sleep(3) # задержка в 3 секунды

Так теперь выглядит первый элемент списка:

In [40]:
news[0]

('https://nplus1.ru/news/2021/05/17/upupa-epops',
 '2021-05-17',
 'Сергей Коленов',
 'Окраска яиц секретом копчиковой железы оказалась признаком, связанным с шансами на появление птенцов.',
 'Удоды охладели к самкам с насыщенно раскрашенными яйцами',
 'Самки удодов окрашивают отложенные яйца секретом копчиковой железы\xa0— он\xa0содержит симбиотические бактерии, которые вырабатывают антибиотики и\xa0защищают развивающийся зародыш от\xa0инфекций. В\xa0результате яйца приобретают зеленовато-серый оттенок. Испанские орнитологи экспериментально продемонстрировали, что самцы придирчиво оценивают цвет кладки\xa0и, если он\xa0слишком насыщенный (это указывает, что концентрация симбиотических бактерий в\xa0секрете низкая), приносят самке меньше корма. Напротив, менее интенсивная окраска заставляет самцов активнее снабжать партнершу пищей. Как отмечается в\xa0статье для журнала Proceedings of\xa0the Royal Society\xa0B, вклад самца зависит от\xa0того, насколько велики шансы на\xa0вылупление здор

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [41]:
import pandas as pd

In [42]:
df = pd.DataFrame(news)

In [43]:
df.head(2)

,0,1,2,3,4,5,6,7
0,https://nplus1.ru/news/2021/05/17/upupa-epops,2021-05-17,Сергей Коленов,Окраска яиц секретом копчиковой железы оказала...,Удоды охладели к самкам с насыщенно раскрашенн...,Самки удодов окрашивают отложенные яйца секрет...,Зоология,3.1
1,https://nplus1.ru/news/2021/05/17/hunting-herd...,2021-05-17,Михаил Подрезов,Первые козы и крупно рогатый скот появились в ...,Древние охотники с кенийского побережья выменя...,Археологи обнаружили самый ранний случай прису...,"Археология,История",2.7


Переименуем столбцы в базе.

In [44]:
df.columns = ['link', 'date', 'author', 'desc', 'title', 'text', 'rubric', 'diffc']

In [45]:
df.head(2)

,link,date,author,desc,title,text,rubric,diffc
0,https://nplus1.ru/news/2021/05/17/upupa-epops,2021-05-17,Сергей Коленов,Окраска яиц секретом копчиковой железы оказала...,Удоды охладели к самкам с насыщенно раскрашенн...,Самки удодов окрашивают отложенные яйца секрет...,Зоология,3.1
1,https://nplus1.ru/news/2021/05/17/hunting-herd...,2021-05-17,Михаил Подрезов,Первые козы и крупно рогатый скот появились в ...,Древние охотники с кенийского побережья выменя...,Археологи обнаружили самый ранний случай прису...,"Археология,История",2.7


Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [46]:
df.to_excel('nplus-news.xlsx')